In [55]:
%esptool erase

Executing:
  esptool.py --port /dev/ttyUSB4 erase_flash

esptool.py v2.0
Connecting....
[Press the PRG button now if required]
Detecting chip type... ESP8266
Chip is ESP8266
Uploading stub...
Running stub...
Stub running...
Erasing flash (this may take a while)...
Chip erase completed successfully in 9.6s
Hard resetting...


In [56]:
%esptool esp8266 /home/julian/extrepositories/micropython/ports/esp8266/build-GENERIC/firmware-combined.bin

Executing:
  esptool.py --port /dev/ttyUSB4 --baud 460800 write_flash --flash_size=detect -fm dio 0 /home/julian/extrepositories/micropython/ports/esp8266/build-GENERIC/firmware-combined.bin

esptool.py v2.0
Connecting....
[Press the PRG button now if required]
Detecting chip type... ESP8266
Chip is ESP8266
Uploading stub...
Running stub...
Stub running...
Changing baud rate to 460800
Changed.
Configuring flash size...
Auto-detected Flash size: 4MB
Flash params set to 0x0240
Compressed 615876 bytes to 401882...
Wrote 615876 bytes (401882 compressed) at 0x00000000 in 9.3 seconds (effective 528.3 kbit/s)...
Hash of data verified.

Leaving...
Hard resetting...


In [30]:
%sendtofile config.txt

wifiname        DoESLiverpool
wifipassword    decafbad00
mqttbroker      10.0.100.1
pinneopixel     15
pinled          2
boardname       neogrid1


Sent 6 lines (146 bytes) to config.txt.


In [40]:
%sendtofile main.py

from mqtt_as import MQTTClient, config
import uasyncio as asyncio
import os, time, machine

fconfig = dict(x.split()  for x in open("config.txt"))
pinled = machine.Pin(int(fconfig["pinled"]), machine.Pin.OUT)
 
import neopixel
np = neopixel.NeoPixel(machine.Pin(int(fconfig["pinneopixel"])), 81)
np.fill((20,0,80))
np.write()

import framebuf
fbuff = framebuf.FrameBuffer(bytearray(9*9*2), 9, 9, framebuf.RGB565)

def chex(su):
    u = int(su, 16)
    return (((u&0xF80000)>>8) | ((u&0x00FC00)>>5) | ((u&0x0000F8)>>3))

def show():
    for iy in range(9):
        oiy = (iy%2)
        fix = (-1 if oiy else 1)
        riy = iy*9 + 8*oiy
        for ix in range(9):
            v = fbuff.pixel(ix, iy)
            r = (v&0b1111100000000000)>>8
            g = (v&0b0000011111100000)>>2
            b = (v&0b0000000000011111)<<3
            np[riy + ix*fix] = (r, g, b) 
    np.write()
    
pinled.value(0)
time.sleep(2)
pinled.value(1)

boardname = fconfig["boardname"]
async def callbackcmdtask(topic, msg):
    print("executing", [msg])
    pinled.value(0)
    try:
        exec(msg)
        await client.publish(boardname+'/reply', "1")
    except Exception as e:
        print(e)
        await client.publish(boardname+'/reply/exception', str(e))
    pinled.value(1)
    return

def callbackcmd(topic, msg, retained):
    aloop.create_task(callbackcmdtask(topic, msg))

async def mainconnect():
    print("connecting")
    fbuff.fill(0x0000); fbuff.text("C", 0, 1, 0xf000); show()
    await client.connect()
    print("subscribing")
    fbuff.fill(0x0000); fbuff.text("S", 0, 1, 0xf000); show()
    await client.subscribe(boardname+'/cmd', 1)
    await client.publish(boardname+"/ip", client._sta_if.ifconfig()[0], retain=True, qos=1)
    fbuff.fill(0x0000); fbuff.text("R", 0, 1, 0xf000); show()
    while True:
        await client.publish(boardname+'/time', str(time.time()))
        await asyncio.sleep_ms(2000)
    
config['subs_cb'] = callbackcmd
config['server'] = fconfig["mqttbroker"]
config['ssid'] = fconfig["wifiname"]
config['wifi_pw'] = fconfig["wifipassword"]

MQTTClient.DEBUG = True
client = MQTTClient(config)

aloop = asyncio.get_event_loop()
aloop.run_until_complete(mainconnect())


Sent 75 lines (2205 bytes) to main.py.


In [ ]:
#      RST          |  GPIO1   TX
#       A0          |  GPIO3   RX
#       D0  GPIO16  |  GPIO5   D1  SCL
# SCK   D5  GPIO14  |  GPIO4   D2  SDA
# MISO  D6  GPIO12  |  GPIO0   D3
# MOSI  D7  GPIO13  |  GPIO2   D4  LED
# SS    D8  GPIO15  |          GND
#      3V3          |          5V


In [1]:
%serialconnect

Connecting to --port=/dev/ttyUSB4 --baud=115200 
Ready.


In [2]:
%sendtofile config.txt

wifiname        DoESLiverpool
wifipassword    decafbad00
mqttbroker      10.0.100.1
pinneopixel     15
pinled          2
boardname       neoring1


Sent 6 lines (146 bytes) to config.txt.


In [3]:
%sendtofile main.py

from mqtt_as import MQTTClient, config
import uasyncio as asyncio
import os, time, machine

fconfig = dict(x.split()  for x in open("config.txt"))
pinled = machine.Pin(int(fconfig["pinled"]), machine.Pin.OUT)
 
import neopixel
np = neopixel.NeoPixel(machine.Pin(int(fconfig["pinneopixel"])), 24)
np.fill((2,1,12))
np.write()

pinled.value(0)
time.sleep(2)
pinled.value(1)

boardname = fconfig["boardname"]
async def callbackcmdtask(topic, msg):
    print("executing", [msg])
    pinled.value(0)
    try:
        exec(msg)
        await client.publish(boardname+'/reply', "1")
    except Exception as e:
        print(e)
        await client.publish(boardname+'/reply/exception', str(e))
    pinled.value(1)
    return

def callbackcmd(topic, msg, retained):
    aloop.create_task(callbackcmdtask(topic, msg))

async def mainconnect():
    print("connecting")
    np.fill((5,0,0)); np.write()
    await client.connect()
    print("subscribing")
    np.fill((0,5,0)); np.write()
    await client.subscribe(boardname+'/cmd', 1)
    await client.publish(boardname+"/ip", client._sta_if.ifconfig()[0], retain=True, qos=1)
    np.fill((2,1,12)); np.write()
    while True:
        await client.publish(boardname+'/time', str(time.time()))
        await asyncio.sleep_ms(2000)
    
config['subs_cb'] = callbackcmd
config['server'] = fconfig["mqttbroker"]
config['ssid'] = fconfig["wifiname"]
config['wifi_pw'] = fconfig["wifipassword"]

MQTTClient.DEBUG = True
client = MQTTClient(config)

aloop = asyncio.get_event_loop()
aloop.run_until_complete(mainconnect())


Sent 55 lines (1556 bytes) to main.py.
